# Apply PSO on Matching Network Optimization
From separate code on PSO and Matching Network, it's time to merge these 2 to make an optimization for MN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

----------------------------------------------------------
## I. Define Matching Network

Still, suppose I have a MN of 5 component like this:

*-------------L2-------------------C3---------------C1---------------*

     |          |             |             |
           
    Z_out      L1             C2             Z_load
           
     |          |             |             |
      
*---------------------------------------------------------------------*

and Z_load is not matched at 50Ohm, so with help from MN, I expect to get Z_out matched

Let's try to simplify the parallel or series type of a circuit

In [2]:
def parallel(Z1, Z2):
    return 1/(1/Z1 + 1/Z2)

def series(Z1, Z2): 
    return Z1 + Z2

Then, I assign type (R/L/C), connection orientation (ser/par), lowest and highest values of 5 components, respectively:

In [3]:
num_component = 5

component_type = ('C', 'C', 'C', 'L', 'L')
component_topology = ('ser', 'par','ser', 'par','ser')
range_C = (1e-12, 100e-12)
range_distance_C = range_C[1] - range_C[0]
range_L = (1e-9, 100e-9)
range_distance_L = range_L[1] - range_L[0]

And the function to calculate output impedance from Z load and MN

*note: for component type: 0 - C, 1 - L, 2 - R; for component placement: 0 - series, 1 - parallel*

In [4]:
def out_impedance(num_component, components, freq, Z_load):
    Z = np.zeros((num_component+1,),dtype = complex)
    Z_out = np.zeros((num_component+1,),dtype = complex)
    # calculate impedance of each component
    Z[0] = Z_load
    #print (Z[0],end = '   ')
    for i in range(num_component):
        if components[i][0] == 0: # C
            Z[i+1] = 1/(1j*2*np.pi*freq*components[i][2])
        elif components[i][0] == 1: # L
            Z[i+1] = 1j*2*np.pi*freq*components[i][2]
        else: # R
            Z[i+1] = components[i][2]
    # calculate result
    Z_out[0] = Z_load
    for i in range(num_component):
        if components[i][1] == 0: # series
            Z_out[i+1] = series(Z_out[i], Z[i+1])
        else:                     # parallel
            Z_out[i+1] = parallel(Z_out[i], Z[i+1])
        #print ('\n Zout', i+1,' :', Z_out[i+1])
    return Z_out[num_component]

---------------------------------------------
## II. Apply PSO
General parameters of PSO are defined here, meaning of them is *PSOsinc* notebook


In [5]:
w = 0.5
c1 = 2
c2 = 2
max_iter = 5000
freq = 1e+9
Z_load = 20
Z0 = 50

### First step
Define the Solution Space

Five bees, or better-5 particles, will be assigned to the task to look for the best position in their so-called Solution Space. This space is 5D, as each agent needs 5 variables to define its location, seem unrealistic hah? :v

In [6]:
agent_quantity = 5
space_dimention = 5
agent_bound = np.zeros([5,2])
for i in range(space_dimention):
    if component_type[i] == 'C':
        agent_bound[i] = range_C
    else:
        agent_bound[i] = range_L
agent_bound

array([[1.e-12, 1.e-10],
       [1.e-12, 1.e-10],
       [1.e-12, 1.e-10],
       [1.e-09, 1.e-07],
       [1.e-09, 1.e-07]])

### Second step
Define a fitness function. In this case, it's the S11 at given frequency

In [7]:
def fitness(agent):
    components = np.zeros([5,3])
    for i in range(agent_quantity):
        # Component type
        if component_type[i] == 'C':
            components[i][0] = 0 # C
        elif component_type[i] == 'L':
            components[i][0] = 1 # L
        else:
            components[i][0] = 1 # R
        # Component placement
        if component_topology[i] == 'ser':
            components[i][1] = 0
        else:
            components[i][1] = 1
        # Component value
        components[i][2] = agent[i]
        
    #print(components)
    Z_out = out_impedance(agent_quantity, components, freq, Z_load)
    fitness = np.abs((Z_out - Z0)/(Z_out + Z0))
    return 1 - fitness
# Check fitness function
fitness([10e-12, 20e-12, 30e-12, 40e-9, 50e-9])

0.0015327902737268806

### Third step 
Initialize Random Swam Location and Velocities

In [8]:
agent_location = np.zeros([agent_quantity, space_dimention])
for i in range(space_dimention):
    if component_type[i] == 'C':
        for j in range(agent_quantity):
            agent_location[j][i] = range_distance_C*np.random.rand()+range_C[0]
    elif component_type[i] == 'L':
        for j in range(agent_quantity):
            agent_location[j][i] = range_distance_L*np.random.rand()+range_L[0]

agent_location

array([[8.59887669e-11, 9.64567348e-11, 7.68048172e-11, 6.58980097e-08,
        1.53568608e-08],
       [6.39097396e-11, 9.72818383e-12, 6.15378137e-11, 3.44585106e-08,
        7.68385558e-08],
       [2.91350695e-11, 7.21259870e-11, 4.68678010e-11, 9.17461465e-08,
        3.59469937e-08],
       [3.82296509e-11, 6.77851866e-12, 2.51937906e-11, 5.70039219e-08,
        2.03026072e-08],
       [9.88205623e-11, 6.43968005e-11, 2.65991789e-12, 3.55904391e-08,
        9.61298576e-08]])

*pbest at first is also the first swam*

In [9]:
agent_pbest = agent_location

In [10]:
agent_velocity = np.zeros([agent_quantity, space_dimention])
for i in range(space_dimention):
    if component_type[i] == 'C':
        for j in range(agent_quantity):
            agent_velocity[j][i] = range_distance_C*np.random.rand()+range_C[0]
    elif component_type[i] == 'L':
        for j in range(agent_quantity):
            agent_velocity[j][i] = range_distance_L*np.random.rand()+range_L[0]

agent_velocity

array([[6.14687341e-11, 9.64183098e-11, 5.66685606e-11, 4.87466330e-08,
        5.78539579e-08],
       [6.71178082e-11, 4.60533810e-11, 7.94325022e-11, 2.78530396e-08,
        4.82183717e-08],
       [2.11740446e-11, 3.67661563e-11, 9.14086519e-11, 6.90731204e-08,
        9.54843650e-08],
       [9.94983142e-11, 7.09935819e-11, 1.01987809e-11, 6.14037408e-08,
        8.44640121e-08],
       [3.14718065e-11, 6.18799426e-11, 4.62251723e-11, 5.74693388e-08,
        9.52294074e-08]])

### Fourth step
Systematically Fly the Particles Through the Solution Space

a) Evaluate the Particles's Fitness, Compare to gbest, pbest

In [11]:
agent_fitness = np.zeros([agent_quantity,1])
pbest = np.zeros([agent_quantity,1])

for i in range(0,agent_quantity):
    agent_fitness[i] = fitness(agent_location[i])
    if agent_fitness[i] > pbest[i]:
            pbest[i] = agent_fitness[i]
            agent_pbest[i] = agent_location[i]
    
pbest

array([[0.00121029],
       [0.0035523 ],
       [0.00042395],
       [0.0681304 ],
       [0.00020806]])

In [12]:
gbest = pbest.max()
gbest_position = pbest.argmax()
print('\n Global best', gbest)
print('\n Global best position', gbest_position)
print('\n Agent at best position', agent_pbest[gbest_position])


 Global best 0.06813040141943372

 Global best position 3

 Agent at best position [3.82296509e-11 6.77851866e-12 2.51937906e-11 5.70039219e-08
 2.03026072e-08]


b) Update the Particle's velocity 

In [13]:
for i in range(agent_quantity):
    agent_velocity[i] = w*agent_velocity[i] + c1*np.random.rand()*(agent_pbest[i] - agent_location[i]) + c2*np.random.rand()*(agent_pbest[gbest_position]- agent_location[i])
    
agent_velocity

array([[-5.00489554e-11, -1.03479253e-10, -5.89644503e-11,
         9.32919462e-09,  3.72925824e-08],
       [ 9.67409784e-12,  2.02832350e-11,  5.91302085e-12,
         3.48957925e-08, -2.84743622e-08],
       [ 2.09365611e-11, -5.59816595e-11,  2.10395281e-11,
        -4.99972923e-09,  2.99390338e-08],
       [ 4.97491571e-11,  3.54967910e-11,  5.09939044e-12,
         3.07018704e-08,  4.22320060e-08],
       [-4.90149204e-11, -3.06341348e-11,  4.71935380e-11,
         5.16183101e-08, -3.34185206e-08]])

c) Move the particle

In [14]:
agent_location = agent_location + agent_velocity
for i in range(agent_quantity):
    for j in range(space_dimention):
        if agent_location[i][j] > agent_bound[j][1]:
            agent_location[i][j] = 2*agent_bound[j][1] - agent_location[i][j]
        if agent_location[i][j] < agent_bound[j][0]:
            agent_location[i][j] = 2*agent_bound[j][0] - agent_location[i][j]
        
agent_location

array([[3.59398115e-11, 9.02251784e-12, 1.78403669e-11, 7.52272043e-08,
        5.26494432e-08],
       [7.35838374e-11, 3.00114188e-11, 6.74508346e-11, 6.93543031e-08,
        4.83641936e-08],
       [5.00716306e-11, 1.61443275e-11, 6.79073291e-11, 8.67464173e-08,
        6.58860275e-08],
       [8.79788080e-11, 4.22753096e-11, 3.02931810e-11, 8.77057923e-08,
        6.25346132e-08],
       [4.98056419e-11, 3.37626657e-11, 4.98534559e-11, 8.72087492e-08,
        6.27113370e-08]])

### Fifth step
Repeat everything in Step 4 untill iteration being out of range 

In [15]:
for k in range(1,max_iter):
    if gbest > 0.99:
        print('Converged!!! Hura!!')
        break
    # Step 4a
    for i in range(agent_quantity):
        agent_fitness[i] = fitness(agent_location[i])
        if agent_fitness[i] > pbest[i]:
            pbest[i] = agent_fitness[i]
            agent_pbest[i] = agent_location[i]
    
    gbest = pbest.max()
    gbest_position = pbest.argmax()
    # Step 4b
    for i in range(agent_quantity):
        agent_velocity[i] = w*agent_velocity[i] + c1*np.random.rand()*(agent_pbest[i] - agent_location[i]) + c2*np.random.rand()*(agent_pbest[gbest_position]- agent_location[i])
    
    # Step 4c
    agent_location = agent_location + agent_velocity
    for i in range(agent_quantity):
        for j in range(space_dimention):
            if agent_location[i][j] > agent_bound[j][1]:
                agent_location[i][j] = 2*agent_bound[j][1] - agent_location[i][j]
            if agent_location[i][j] < agent_bound[j][0]:
                agent_location[i][j] = 2*agent_bound[j][0] - agent_location[i][j]
    print('Iteration: ', k)
    print('Gbest in this iteration: ', gbest)
                
            
gbest

Iteration:  1
Gbest in this iteration:  0.06813040141943372
Iteration:  2
Gbest in this iteration:  0.1434323406777268
Iteration:  3
Gbest in this iteration:  0.1965082678496446
Iteration:  4
Gbest in this iteration:  0.9283389691728087
Iteration:  5
Gbest in this iteration:  0.9283389691728087
Iteration:  6
Gbest in this iteration:  0.9283389691728087
Iteration:  7
Gbest in this iteration:  0.9283389691728087
Iteration:  8
Gbest in this iteration:  0.9283389691728087
Iteration:  9
Gbest in this iteration:  0.9283389691728087
Iteration:  10
Gbest in this iteration:  0.9283389691728087
Iteration:  11
Gbest in this iteration:  0.9467492288092638
Iteration:  12
Gbest in this iteration:  0.9467492288092638
Iteration:  13
Gbest in this iteration:  0.9638544561280198
Iteration:  14
Gbest in this iteration:  0.9638544561280198
Iteration:  15
Gbest in this iteration:  0.9660034807758913
Iteration:  16
Gbest in this iteration:  0.9660034807758913
Iteration:  17
Gbest in this iteration:  0.96600

Iteration:  334
Gbest in this iteration:  0.9897512210447659
Iteration:  335
Gbest in this iteration:  0.9897512210447663
Iteration:  336
Gbest in this iteration:  0.9897512210447668
Iteration:  337
Gbest in this iteration:  0.989751221044767
Iteration:  338
Gbest in this iteration:  0.9897512210447671
Iteration:  339
Gbest in this iteration:  0.9897512210447676
Iteration:  340
Gbest in this iteration:  0.9897512210447681
Iteration:  341
Gbest in this iteration:  0.9897512210447681
Iteration:  342
Gbest in this iteration:  0.9897512210447688
Iteration:  343
Gbest in this iteration:  0.989751221044769
Iteration:  344
Gbest in this iteration:  0.9897512210447695
Iteration:  345
Gbest in this iteration:  0.9897512210447702
Iteration:  346
Gbest in this iteration:  0.9897512210447711
Iteration:  347
Gbest in this iteration:  0.9897512210447715
Iteration:  348
Gbest in this iteration:  0.9897512210447722
Iteration:  349
Gbest in this iteration:  0.9897512210447726
Iteration:  350
Gbest in t

Iteration:  705
Gbest in this iteration:  0.9897512210448084
Iteration:  706
Gbest in this iteration:  0.9897512210448084
Iteration:  707
Gbest in this iteration:  0.9897512210448084
Iteration:  708
Gbest in this iteration:  0.9897512210448084
Iteration:  709
Gbest in this iteration:  0.9897512210448084
Iteration:  710
Gbest in this iteration:  0.9897512210448084
Iteration:  711
Gbest in this iteration:  0.9897512210448084
Iteration:  712
Gbest in this iteration:  0.9897512210448084
Iteration:  713
Gbest in this iteration:  0.9897512210448084
Iteration:  714
Gbest in this iteration:  0.9897512210448084
Iteration:  715
Gbest in this iteration:  0.9897512210448084
Iteration:  716
Gbest in this iteration:  0.9897512210448084
Iteration:  717
Gbest in this iteration:  0.9897512210448084
Iteration:  718
Gbest in this iteration:  0.9897512210448084
Iteration:  719
Gbest in this iteration:  0.9897512210448084
Iteration:  720
Gbest in this iteration:  0.9897512210448084
Iteration:  721
Gbest in

Iteration:  1004
Gbest in this iteration:  0.9897512210448084
Iteration:  1005
Gbest in this iteration:  0.9897512210448084
Iteration:  1006
Gbest in this iteration:  0.9897512210448084
Iteration:  1007
Gbest in this iteration:  0.9897512210448084
Iteration:  1008
Gbest in this iteration:  0.9897512210448084
Iteration:  1009
Gbest in this iteration:  0.9897512210448084
Iteration:  1010
Gbest in this iteration:  0.9897512210448084
Iteration:  1011
Gbest in this iteration:  0.9897512210448084
Iteration:  1012
Gbest in this iteration:  0.9897512210448084
Iteration:  1013
Gbest in this iteration:  0.9897512210448084
Iteration:  1014
Gbest in this iteration:  0.9897512210448084
Iteration:  1015
Gbest in this iteration:  0.9897512210448084
Iteration:  1016
Gbest in this iteration:  0.9897512210448084
Iteration:  1017
Gbest in this iteration:  0.9897512210448084
Iteration:  1018
Gbest in this iteration:  0.9897512210448084
Iteration:  1019
Gbest in this iteration:  0.9897512210448084
Iteratio

Iteration:  1369
Gbest in this iteration:  0.9897512210448137
Iteration:  1370
Gbest in this iteration:  0.9897512210448137
Iteration:  1371
Gbest in this iteration:  0.9897512210448137
Iteration:  1372
Gbest in this iteration:  0.9897512210448137
Iteration:  1373
Gbest in this iteration:  0.9897512210448137
Iteration:  1374
Gbest in this iteration:  0.9897512210448137
Iteration:  1375
Gbest in this iteration:  0.9897512210448137
Iteration:  1376
Gbest in this iteration:  0.9897512210448137
Iteration:  1377
Gbest in this iteration:  0.989751221044814
Iteration:  1378
Gbest in this iteration:  0.9897512210448144
Iteration:  1379
Gbest in this iteration:  0.9897512210448144
Iteration:  1380
Gbest in this iteration:  0.9897512210448144
Iteration:  1381
Gbest in this iteration:  0.9897512210448144
Iteration:  1382
Gbest in this iteration:  0.9897512210448144
Iteration:  1383
Gbest in this iteration:  0.9897512210448144
Iteration:  1384
Gbest in this iteration:  0.9897512210448144
Iteration

...

Iteration:  3628
Gbest in this iteration:  0.9897512210449286
Iteration:  3629
Gbest in this iteration:  0.9897512210449286
Iteration:  3630
Gbest in this iteration:  0.9897512210449286
Iteration:  3631
Gbest in this iteration:  0.9897512210449286
Iteration:  3632
Gbest in this iteration:  0.9897512210449286
Iteration:  3633
Gbest in this iteration:  0.9897512210449286
Iteration:  3634
Gbest in this iteration:  0.9897512210449286
Iteration:  3635
Gbest in this iteration:  0.9897512210449286
Iteration:  3636
Gbest in this iteration:  0.9897512210449286
Iteration:  3637
Gbest in this iteration:  0.9897512210449286
Iteration:  3638
Gbest in this iteration:  0.9897512210449286
Iteration:  3639
Gbest in this iteration:  0.9897512210449286
Iteration:  3640
Gbest in this iteration:  0.9897512210449286
Iteration:  3641
Gbest in this iteration:  0.9897512210449286
Iteration:  3642
Gbest in this iteration:  0.9897512210449286
Iteration:  3643
Gbest in this iteration:  0.9897512210449286
Iteratio

Iteration:  4065
Gbest in this iteration:  0.9897512210449286
Iteration:  4066
Gbest in this iteration:  0.9897512210449286
Iteration:  4067
Gbest in this iteration:  0.9897512210449286
Iteration:  4068
Gbest in this iteration:  0.9897512210449286
Iteration:  4069
Gbest in this iteration:  0.9897512210449286
Iteration:  4070
Gbest in this iteration:  0.9897512210449286
Iteration:  4071
Gbest in this iteration:  0.9897512210449286
Iteration:  4072
Gbest in this iteration:  0.9897512210449286
Iteration:  4073
Gbest in this iteration:  0.9897512210449286
Iteration:  4074
Gbest in this iteration:  0.9897512210449286
Iteration:  4075
Gbest in this iteration:  0.9897512210449286
Iteration:  4076
Gbest in this iteration:  0.9897512210449286
Iteration:  4077
Gbest in this iteration:  0.9897512210449286
Iteration:  4078
Gbest in this iteration:  0.9897512210449286
Iteration:  4079
Gbest in this iteration:  0.9897512210449286
Iteration:  4080
Gbest in this iteration:  0.9897512210449286
Iteratio

Iteration:  4406
Gbest in this iteration:  0.9897512210449286
Iteration:  4407
Gbest in this iteration:  0.9897512210449286
Iteration:  4408
Gbest in this iteration:  0.9897512210449286
Iteration:  4409
Gbest in this iteration:  0.9897512210449286
Iteration:  4410
Gbest in this iteration:  0.9897512210449286
Iteration:  4411
Gbest in this iteration:  0.9897512210449286
Iteration:  4412
Gbest in this iteration:  0.9897512210449286
Iteration:  4413
Gbest in this iteration:  0.9897512210449286
Iteration:  4414
Gbest in this iteration:  0.9897512210449286
Iteration:  4415
Gbest in this iteration:  0.9897512210449286
Iteration:  4416
Gbest in this iteration:  0.9897512210449286
Iteration:  4417
Gbest in this iteration:  0.9897512210449286
Iteration:  4418
Gbest in this iteration:  0.9897512210449286
Iteration:  4419
Gbest in this iteration:  0.9897512210449286
Iteration:  4420
Gbest in this iteration:  0.9897512210449286
Iteration:  4421
Gbest in this iteration:  0.9897512210449286
Iteratio

Iteration:  4737
Gbest in this iteration:  0.9897512210449286
Iteration:  4738
Gbest in this iteration:  0.9897512210449286
Iteration:  4739
Gbest in this iteration:  0.9897512210449286
Iteration:  4740
Gbest in this iteration:  0.9897512210449286
Iteration:  4741
Gbest in this iteration:  0.9897512210449286
Iteration:  4742
Gbest in this iteration:  0.9897512210449286
Iteration:  4743
Gbest in this iteration:  0.9897512210449286
Iteration:  4744
Gbest in this iteration:  0.9897512210449286
Iteration:  4745
Gbest in this iteration:  0.9897512210449286
Iteration:  4746
Gbest in this iteration:  0.9897512210449286
Iteration:  4747
Gbest in this iteration:  0.9897512210449286
Iteration:  4748
Gbest in this iteration:  0.9897512210449286
Iteration:  4749
Gbest in this iteration:  0.9897512210449286
Iteration:  4750
Gbest in this iteration:  0.9897512210449286
Iteration:  4751
Gbest in this iteration:  0.9897512210449286
Iteration:  4752
Gbest in this iteration:  0.9897512210449286
Iteratio

0.9897512210449286

-----------------------------------------------------------
# III. Result
Final result is shown here, good or not, it've already taken a lot of time

In [16]:
print('\n Agent at best position', agent_pbest[gbest_position])


 Agent at best position [1.98933523e-11 4.99596802e-12 1.36157120e-11 5.74629894e-09
 4.05076361e-09]


In [17]:
agent_gbest = agent_pbest[gbest_position]
components = np.zeros([5,3])
for i in range(agent_quantity):
    # Component type
    if component_type[i] == 'C':
        components[i][0] = 0 # C
    elif component_type[i] == 'L':
        components[i][0] = 1 # L
    else:
        components[i][0] = 1 # R
    # Component placement
    if component_topology[i] == 'ser':
        components[i][1] = 0
    else:
        components[i][1] = 1
    # Component value
    components[i][2] = agent_gbest[i]
        
#print(components)
Z_out = out_impedance(agent_quantity, components, freq, Z_load)
print('Z output is ', Z_out)

Z output is  (49.172506357465764-0.5902151461083278j)
